In [ ]:
fastfood_prices = {
    "Coke": 1.99,
    "Fries": 2.49,
    "Hamburger": 3.99,
    "Cheeseburger": 4.49,
    "Chicken Nuggets": 4.99,
    "Ice Cream Sundae": 2.99,
    "Salad": 3.99,
    "Fishburger": 4.99,
    "Hot Dog": 3.49,
    "Milkshake": 2.99
}
product_names_string = ', '.join(fastfood_prices.keys())

In [ ]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

client = OpenAI()

audio_file = open("speech.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)

In [ ]:
def calculate_total_price(**kwargs):
    products_with_quantities = kwargs.get('products_with_quantities', [])
    total_price = sum(fastfood_prices[product['name']] * product['quantity'] for product in products_with_quantities)
    return total_price


In [ ]:
products_to_buy = {"products_with_quantities": [{"name": "Coke", "quantity": 1}, {"name": "Hot Dog", "quantity": 1}]}

final_price = calculate_total_price(**products_to_buy)
print(f"Der Gesamtpreis beträgt: {final_price}€")

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "calculate_total_price",
        "description": "Calculates the total price for a list of products with specified quantities, based on a predefined price list.",
        "parameters": {
            "type": "object",
            "properties": {
                "products_with_quantities": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "The name of the product."
                            },
                            "quantity": {
                                "type": "integer",
                                "description": "The quantity of the product."
                            }
                        },
                        "required": ["name", "quantity"]
                    },
                    "description": "A list of products with their names and quantities. Each item is an object with 'name' and 'quantity' properties."
                }
            },
            "required": ["products_with_quantities"]
        }
    }
}]


In [ ]:
import json
from openai import OpenAI

GPT_MODEL = "gpt-3.5-turbo-0613"
client = OpenAI()

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": f"You are an assistant to reformulate textinut of a user. He orders products. You identify the following products {product_names_string} in the request and rename it if there is an error. Example: 'hat dog' -> 'Hot Dog' Also reformulate or create numbers from the request. Example: 'A Hot Dog' -> '1 Hot Dog'"},
    {"role": "user", "content": "I want to order 1 coke and one Hot dag!"}
  ],
)
new_question = completion.choices[0].message
print(new_question)

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "I want to order 1 coke and one Hot Dog!"}
  ],
  tools=tools
)

print(completion.choices[0].message)

In [ ]:
message = completion.choices[0].message
message

In [ ]:
print(message.tool_calls[0].function.name)
print(message.tool_calls[0].function.arguments)

In [ ]:
import json

arguments = json.loads(message.tool_calls[0].function.arguments)

In [ ]:
final_price = calculate_total_price(**arguments)

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": f"You are a helpful assistant. You tell the client 'Thanks for your ordner. Please drive to the issuing counter and put the money in the machine. The final price is: {final_price}€"},
  ],
)
message = completion.choices[0].message
message

In [ ]:
from pathlib import Path
from openai import OpenAI
import os

client = OpenAI()

speech_file_path = os.path.join(os.getcwd(), "order.mp3")
print(speech_file_path)
response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
input=message.content
)
response.stream_to_file(speech_file_path)